<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/finetuning/mistralai_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine Tuning MistralAI models using Finetuning API

In this notebook, we walk through an example of fine-tuning `open-mistral-7b` using MistralAI finetuning API.

Specifically, we attempt to distill `mistral-large-latest`'s knowledge, by generating training data with `mistral-large-latest` to then fine-tune `open-mistral-7b`.

All training data is generated using two different sections of our index data, creating both a training and evalution set.

We will use `mistral-small-largest` to create synthetic training and evaluation questions to avoid any biases towards `open-mistral-7b` and `mistral-large-latest`.

We then finetune with our `MistraAIFinetuneEngine` wrapper abstraction.

Evaluation is done using the `ragas` library, which we will detail later on.

We can monitor the metrics on `Weights & Biases`

In [ ]:
%pip install llama-index-finetuning
%pip install llama-index-finetuning-callbacks
%pip install llama-index-llms-mistralai
%pip install llama-index-embeddings-mistralai

In [ ]:
# !pip install llama-index pypdf sentence-transformers ragas

In [1]:
# NESTED ASYNCIO LOOP NEEDED TO RUN ASYNC IN A NOTEBOOK
import nest_asyncio

nest_asyncio.apply()

## Set API Key

In [2]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
os.environ["MISTRAL_API_KEY"] = os.getenv("MISTRAL_API_KEY")

## Download Data

Here, we first down load the PDF that we will use to generate training data.

In [6]:
!curl https://www.ipcc.ch/report/ar6/wg2/downloads/report/IPCC_AR6_WGII_Chapter03.pdf --output IPCC_AR6_WGII_Chapter03.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 20.7M  100 20.7M    0     0  24.7M      0 --:--:-- --:--:-- --:--:-- 24.7M


The next step is generating a training and eval dataset.

We will generate 40 training and 40 evaluation questions on different sections of the PDF we downloaded.

We can use `open-mistral-7b` on the eval questions to get our baseline performance.

Then, we will use `mistral-large-latest` on the train questions to generate our training data. 


## Load Data

In [3]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.evaluation import DatasetGenerator

documents = SimpleDirectoryReader(
    input_files=["IPCC_AR6_WGII_Chapter03.pdf"]
).load_data()

## Setup LLM and Embedding Model

In [4]:
from llama_index.llms.mistralai import MistralAI
from llama_index.embeddings.mistralai import MistralAIEmbedding

open_mistral = MistralAI(
    model="open-mistral-7b", temperature=0.1
)  # model to be finetuning
mistral_small = MistralAI(
    model="mistral-small-latest", temperature=0.1
)  # model for question generation
embed_model = MistralAIEmbedding()

## Training and Evaluation Data Generation

In [5]:
question_gen_query = (
    "You are a Teacher/ Professor. Your task is to setup "
    "a quiz/examination. Using the provided context, formulate "
    "a single question that captures an important fact from the "
    "context. Restrict the question to the context information provided."
    "You should generate only question and nothing else."
)

dataset_generator = DatasetGenerator.from_documents(
    documents[:80],
    question_gen_query=question_gen_query,
    llm=mistral_small,
)

/Users/linghuang/miniconda3/envs/mistral/lib/python3.10/site-packages/llama_index/core/evaluation/dataset_generation.py:215: DeprecationWarning: Call to deprecated class DatasetGenerator. (Deprecated in favor of `RagDatasetGenerator` which should be used instead.)
  return cls(


We will generate 40 training and 40 evaluation questions

In [13]:
# Note: This might take sometime.
questions = dataset_generator.generate_questions_from_nodes(num=40)
print("Generated ", len(questions), " questions")

Generated  40  questions


/Users/linghuang/miniconda3/envs/mistral/lib/python3.10/site-packages/llama_index/core/evaluation/dataset_generation.py:312: DeprecationWarning: Call to deprecated class QueryResponseDataset. (Deprecated in favor of `LabelledRagDataset` which should be used instead.)
  return QueryResponseDataset(queries=queries, responses=responses_dict)


In [14]:
questions[10:15]

['What is the estimated relative human dependence on marine ecosystems for coastal protection, nutrition, fisheries economic benefits, and overall, as depicted in Figure 3.1?',
 'What are the limitations of the overall index mentioned in the context, and how were values for reference regions computed?',
 'What are the primary non-climate drivers that alter marine ecosystems and their services, as mentioned in the context?',
 'What are the different lines of evidence used by IPCC reports to assess climate-driven change in ocean and coastal ecosystems?',
 'What new findings from Section 3.3.4 of the IPCC AR6 WGII Chapter 03 PDF document support the previous assessments about the limitations of adaptation options for eukaryotic organisms in response to climate change?']

In [15]:
with open("train_questions.txt", "w") as f:
    for question in questions:
        f.write(question + "\n")

Now, lets generate questions on a completely different set of documents, in order to create our eval dataset.

In [16]:
dataset_generator = DatasetGenerator.from_documents(
    documents[80:],
    question_gen_query=question_gen_query,
    llm=mistral_small,
)

/Users/linghuang/miniconda3/envs/mistral/lib/python3.10/site-packages/llama_index/core/evaluation/dataset_generation.py:215: DeprecationWarning: Call to deprecated class DatasetGenerator. (Deprecated in favor of `RagDatasetGenerator` which should be used instead.)
  return cls(


In [17]:
# Note: This might take sometime.
questions = dataset_generator.generate_questions_from_nodes(num=40)
print("Generated ", len(questions), " questions")

Generated  40  questions


/Users/linghuang/miniconda3/envs/mistral/lib/python3.10/site-packages/llama_index/core/evaluation/dataset_generation.py:312: DeprecationWarning: Call to deprecated class QueryResponseDataset. (Deprecated in favor of `LabelledRagDataset` which should be used instead.)
  return QueryResponseDataset(queries=queries, responses=responses_dict)


In [18]:
with open("eval_questions.txt", "w") as f:
    for question in questions:
        f.write(question + "\n")

## Initial Eval with `open-mistral-7b` Query Engine

For this eval, we will be using the [`ragas` evaluation library](https://github.com/explodinggradients/ragas).

Ragas has a ton of evaluation metrics for RAG pipelines, and you can read about them [here](https://github.com/explodinggradients/ragas/blob/main/docs/metrics.md).

For this notebook, we will be using the following two metrics

- `answer_relevancy` - This measures how relevant is the generated answer to the prompt. If the generated answer is incomplete or contains redundant information the score will be low. This is quantified by working out the chance of an LLM generating the given question using the generated answer. Values range (0,1), higher the better.
- `faithfulness` - This measures the factual consistency of the generated answer against the given context. This is done using a multi step paradigm that includes creation of statements from the generated answer followed by verifying each of these statements against the context. The answer is scaled to (0,1) range. Higher the better.

In [5]:
questions = []
with open("eval_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

In [6]:
questions[:2]

['What were the substantial economic losses in the North American Pacific Coast shellfish aquaculture industry in the 2000s due to ocean acidification, as assessed in SROCC (Bindoff et al., 2019a) and WGII AR5 (Pörtner et al., 2014)?',
 'What are the expected impacts of climate-related ocean changes and human activities on coastal ecosystems, as mentioned in the context?']

In [7]:
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.mistralai import MistralAIEmbedding

# limit the context window to 2048 tokens so that refine is used
from llama_index.core import Settings

Settings.context_window = 2048
Settings.llm = open_mistral
Settings.embed_model = MistralAIEmbedding()

index = VectorStoreIndex.from_documents(
    documents,
)

query_engine = index.as_query_engine(similarity_top_k=2)

In [8]:
contexts = []
answers = []

for question in questions:
    response = query_engine.query(question)
    contexts.append([x.node.get_content() for x in response.source_nodes])
    answers.append(str(response))

In [16]:
# with open("contexts.txt", "w") as f:
#     for context in contexts:
#         f.write(str(context) + "\n")
        
# with open("answers.txt", "w") as f:
#     for answer in answers:
#         f.write(answer + "\n")    

In [9]:
# We will use OpenAI LLM for evaluation using RAGAS
import openai
openai.api_key = os.environ["OPENAI_API_KEY"]

# from llama_index.llms.openai import OpenAI
# from llama_index.embeddings.openai import OpenAIEmbedding

# embeddings = OpenAIEmbedding()
# evaluator_llm = OpenAI(model="gpt-4o")

In [10]:
from datasets import Dataset

ds = Dataset.from_dict(
    {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
    }
)

/Users/linghuang/miniconda3/envs/mistral/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness

result = evaluate(ds, [answer_relevancy, faithfulness])

Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [02:21<00:00,  1.77s/it]


In [12]:
# from ragas import evaluate
# from ragas.metrics import answer_relevancy, faithfulness

# vector_index = VectorStoreIndex.from_documents(documents)
# query_engine = vector_index.as_query_engine()

# result = evaluate(
#     query_engine=query_engine,
#     metrics=metrics,
#     dataset=ds_dict,
#     llm=evaluator_llm,
#     embeddings=OpenAIEmbedding(),
# )

Let's check the results before finetuning.

In [13]:
print(result)

{'answer_relevancy': 0.7947, 'faithfulness': 0.7851}


## `mistral-large-latest` to Collect Training Data

Here, we use `mistral-large-latest` to collect data that we want `open-mistral-7b` to finetune on.

In [14]:
from llama_index.llms.mistralai import MistralAI
from llama_index.finetuning.callbacks import MistralAIFineTuningHandler
from llama_index.core.callbacks import CallbackManager

finetuning_handler = MistralAIFineTuningHandler()
callback_manager = CallbackManager([finetuning_handler])

llm = MistralAI(model="mistral-large-latest", temperature=0.1)
llm.callback_manager = callback_manager

In [15]:
questions = []
with open("train_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

In [16]:
from llama_index.core import VectorStoreIndex

Settings.embed_model = MistralAIEmbedding()
Settings.llm = llm

index = VectorStoreIndex.from_documents(
    documents,
)

query_engine = index.as_query_engine(similarity_top_k=2)

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mist

In [17]:
from tqdm import tqdm

contexts = []
answers = []

for question in tqdm(questions, desc="Processing questions"):
    response = query_engine.query(question)
    contexts.append(
        "\n".join([x.node.get_content() for x in response.source_nodes])
    )
    answers.append(str(response))

Processing questions:   0%|                                                                                                                                                                                                | 0/40 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


Processing questions:   2%|████▌                                                                                                                                                                                   | 1/40 [00:01<01:12,  1.86s/it]

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


Processing questions:   5%|█████████▏                                                                                                                                                                              | 2/40 [00:09<03:26,  5.43s/it]

...

Processing questions:  95%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊         | 38/40 [03:07<00:09,  4.50s/it]

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


Processing questions:  98%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍    | 39/40 [03:10<00:04,  4.07s/it]

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


Processing questions: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [03:13<00:00,  4.85s/it]


In [18]:
import json
from typing import List


def convert_data_jsonl_format(
    questions: List[str],
    contexts: List[str],
    answers: List[str],
    output_file: str,
) -> None:
    with open(output_file, "w") as outfile:
        for context, question, answer in zip(contexts, questions, answers):
            message_dict = {
                "messages": [
                    {
                        "role": "assistant",
                        "content": "You are a helpful assistant to answer user queries based on provided context.",
                    },
                    {
                        "role": "user",
                        "content": f"context: {context} \n\n question: {question}",
                    },
                    {"role": "assistant", "content": answer},
                ]
            }
            # Write the JSON object in a single line
            json.dump(message_dict, outfile)
            # Add a newline character after each JSON object
            outfile.write("\n")

In [19]:
convert_data_jsonl_format(questions, contexts, answers, "training.jsonl")

## Create `MistralAIFinetuneEngine`

We create an `MistralAIFinetuneEngine`: the finetune engine will take care of launching a finetuning job, and returning an LLM model that you can directly plugin to the rest of LlamaIndex workflows.

We use the default constructor, but we can also directly pass in our finetuning_handler into this engine with the `from_finetuning_handler` class method.

In [22]:
import os
from mistralai.client import MistralClient

api_key = os.environ.get("MISTRAL_API_KEY")
client = MistralClient(api_key=api_key)

with open("training.jsonl", "rb") as f:
    training_data = client.files.create(file=("training_file.jsonl", f))    

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/files "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/files "HTTP/1.1 200 OK"


In [23]:
from mistralai.models.jobs import TrainingParameters

created_jobs = client.jobs.create(
    model="open-mistral-7b",
    training_files=[training_data.id],
    hyperparameters=TrainingParameters(
        training_steps=10,
        learning_rate=0.0001,
        )
)
created_jobs

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/fine_tuning/jobs?dry_run=false "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/fine_tuning/jobs?dry_run=false "HTTP/1.1 200 OK"


Job(id='4a72ae5d-d7d9-4952-ae08-32a2077c26e7', hyperparameters=TrainingParameters(training_steps=10, learning_rate=0.0001), fine_tuned_model=None, model='open-mistral-7b', status='QUEUED', job_type='FT', created_at=1719169309, modified_at=1719169309, training_files=['58e910c9-1806-47b2-91f4-7994ec2af48c'], validation_files=[], object='job', integrations=[])

In [25]:
from llama_index.llms.mistralai import MistralAI
from llama_index.finetuning.callbacks import MistralAIFineTuningHandler
from llama_index.core.callbacks import CallbackManager
from llama_index.finetuning.mistralai import MistralAIFinetuneEngine

finetuning_engine = MistralAIFinetuneEngine(
    base_model="open-mistral-7b",
    training_path="training.jsonl",
    # validation_path="<validation file>", # validation file is optional
    verbose=True,
    training_steps=5,
    learning_rate=0.0001,
)

In [24]:
# # List jobs
# jobs = client.jobs.list()
# print(jobs)

INFO:httpx:HTTP Request: GET https://api.mistral.ai/v1/fine_tuning/jobs?page=0&page_size=10 "HTTP/1.1 200 OK"
HTTP Request: GET https://api.mistral.ai/v1/fine_tuning/jobs?page=0&page_size=10 "HTTP/1.1 200 OK"
data=[Job(id='4a72ae5d-d7d9-4952-ae08-32a2077c26e7', hyperparameters=TrainingParameters(training_steps=10, learning_rate=0.0001), fine_tuned_model=None, model='open-mistral-7b', status='RUNNING', job_type='FT', created_at=1719169309, modified_at=1719169310, training_files=['58e910c9-1806-47b2-91f4-7994ec2af48c'], validation_files=[], object='job', integrations=[])] object='list'


In [ ]:
# from mistralai.models.chat_completion import ChatMessage

# chat_response = client.chat(
#     model=retrieved_job.fine_tuned_model,
#     messages=[ChatMessage(role='user', content='What is the best French cheese?')]
# )

In [26]:
# starts the finetuning of open-mistral-7b

finetuning_engine.finetune()

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/files "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/files "HTTP/1.1 200 OK"
INFO:llama_index.finetuning.mistralai.base:File uploaded...
File uploaded...
File uploaded...
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/fine_tuning/jobs?dry_run=false "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/fine_tuning/jobs?dry_run=false "HTTP/1.1 200 OK"
INFO:llama_index.finetuning.mistralai.base:Training job 8978427e-6196-4a5e-b361-f7dc7662b728 launched. 
Training job 8978427e-6196-4a5e-b361-f7dc7662b728 launched. 
Training job 8978427e-6196-4a5e-b361-f7dc7662b728 launched. 


In [27]:
# This will show the current status of the job - 'RUNNING'
finetuning_engine.get_current_job()

INFO:httpx:HTTP Request: GET https://api.mistral.ai/v1/fine_tuning/jobs/8978427e-6196-4a5e-b361-f7dc7662b728 "HTTP/1.1 200 OK"
HTTP Request: GET https://api.mistral.ai/v1/fine_tuning/jobs/8978427e-6196-4a5e-b361-f7dc7662b728 "HTTP/1.1 200 OK"


DetailedJob(id='8978427e-6196-4a5e-b361-f7dc7662b728', hyperparameters=TrainingParameters(training_steps=5, learning_rate=0.0001), fine_tuned_model=None, model='open-mistral-7b', status='RUNNING', job_type='FT', created_at=1719169494, modified_at=1719169498, training_files=['2a257c03-5c2c-4501-9b7e-4a8d94581494'], validation_files=[], object='job', integrations=[], events=[Event(name='status-updated', data={'status': 'RUNNING'}, created_at=1719169498), Event(name='status-updated', data={'status': 'QUEUED'}, created_at=1719169494)], checkpoints=[], estimated_start_time=None)

In [32]:
# This will show the current status of the job - 'SUCCESS'
finetuning_engine.get_current_job()

INFO:httpx:HTTP Request: GET https://api.mistral.ai/v1/fine_tuning/jobs/8978427e-6196-4a5e-b361-f7dc7662b728 "HTTP/1.1 200 OK"
HTTP Request: GET https://api.mistral.ai/v1/fine_tuning/jobs/8978427e-6196-4a5e-b361-f7dc7662b728 "HTTP/1.1 200 OK"


DetailedJob(id='8978427e-6196-4a5e-b361-f7dc7662b728', hyperparameters=TrainingParameters(training_steps=5, learning_rate=0.0001), fine_tuned_model='ft:open-mistral-7b:1e6a8bb8:20240623:8978427e', model='open-mistral-7b', status='SUCCESS', job_type='FT', created_at=1719169494, modified_at=1719169553, training_files=['2a257c03-5c2c-4501-9b7e-4a8d94581494'], validation_files=[], object='job', integrations=[], events=[Event(name='status-updated', data={'status': 'SUCCESS'}, created_at=1719169553), Event(name='status-updated', data={'status': 'RUNNING'}, created_at=1719169498), Event(name='status-updated', data={'status': 'QUEUED'}, created_at=1719169494)], checkpoints=[], estimated_start_time=None)

In [33]:
ft_llm = finetuning_engine.get_finetuned_model(temperature=0.1)

INFO:httpx:HTTP Request: GET https://api.mistral.ai/v1/fine_tuning/jobs/8978427e-6196-4a5e-b361-f7dc7662b728 "HTTP/1.1 200 OK"
HTTP Request: GET https://api.mistral.ai/v1/fine_tuning/jobs/8978427e-6196-4a5e-b361-f7dc7662b728 "HTTP/1.1 200 OK"
INFO:llama_index.finetuning.mistralai.base:status of the job_id: 8978427e-6196-4a5e-b361-f7dc7662b728 is SUCCESS
status of the job_id: 8978427e-6196-4a5e-b361-f7dc7662b728 is SUCCESS


## Evaluation

Once the finetuned model is created, the next step is running our fine-tuned model on our eval dataset again to measure any performance increase.

In [34]:
from llama_index.core import Settings
from llama_index.llms.mistralai import MistralAI
from llama_index.embeddings.mistralai import MistralAIEmbedding

# Setting up finetuned llm
Settings.llm = ft_llm
Settings.context_window = (
    2048  # limit the context window artifically to test refine process
)
Settings.embed_model = MistralAIEmbedding()

In [35]:
questions = []
with open("eval_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

In [36]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine(similarity_top_k=2, llm=ft_llm)

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mist

In [37]:
contexts = []
answers = []

for question in tqdm(questions, desc="Processing Questions"):
    response = query_engine.query(question)
    contexts.append([x.node.get_content() for x in response.source_nodes])
    answers.append(str(response))

Processing Questions:   0%|                                                                                                                                                                                                | 0/40 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


Processing Questions:   2%|████▌                                                                                                                                                                                   | 1/40 [00:04<03:00,  4.62s/it]

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


Processing Questions:   5%|█████████▏                                                                                                                                                                              | 2/40 [00:09<03:04,  4.84s/it]

...

Processing Questions:  95%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊         | 38/40 [02:27<00:05,  2.94s/it]

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


Processing Questions:  98%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍    | 39/40 [02:30<00:02,  2.89s/it]

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


Processing Questions: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [02:32<00:00,  3.82s/it]


In [38]:
ds = Dataset.from_dict(
    {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
    }
)

result = evaluate(ds, [answer_relevancy, faithfulness])

Evaluating:   0%|                                                                                                                                                                                                          | 0/80 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Evaluating:   1%|██▍                                                                                                                                                                                               | 1/80 [00:01<01:46,  1.35s/it]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/comp

Evaluating:   2%|████▊                                                                                                                                                                                             | 2/80 [00:03<02:41,  2.07s/it]

...

Evaluating:  98%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏    | 78/80 [02:13<00:09,  4.72s/it]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Evaluating:  99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌  | 79/80 [02:14<00:03,  3.65s/it]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [02:21<00:00,  1.77s/it]


Let's check the results with finetuned model

In [39]:
print(result)

{'answer_relevancy': 0.7622, 'faithfulness': 0.8436}


## Observation:

`open-mistral-7b` : 'answer_relevancy': **0.8151**, 'faithfulness': **0.8360**

`open-mistral-7b-finetuned` : 'answer_relevancy': **0.8016**, 'faithfulness': **0.8924**

As you can see there is an increase in faithfulness score and small drop in answer relevancy.

## Exploring Differences

Let's quickly compare the differences in responses, to demonstrate that fine tuning did indeed change something.

In [40]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents)

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mist

In [41]:
questions = []
with open("eval_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

In [42]:
print(questions[20])

What are some examples of economic losses due to disruptions in ocean and coastal ecosystem services partly attributable to climate change, as mentioned in the context?


### Original `open-mistral-7b  `

In [44]:
from llama_index.core.response.notebook_utils import display_response

In [45]:
query_engine = index.as_query_engine(llm=open_mistral)

response = query_engine.query(questions[20])

display_response(response)

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


**`Final Response:`** Economic losses due to disruptions in ocean and coastal ecosystem services partly attributable to climate change, as mentioned in the context, include:

1. The estimated loss of 800 million USD in the farmed-salmon industry in Chile due to a climate-driven harmful algal bloom.
2. Significant economic losses for fishers and service industries supporting commercial and recreational fishing in the USA, following a climate-driven algal bloom, which harmed 84% of surveyed residents from 16 California coastal communities.
3. Disruptions to subsistence and recreational fishing for razor clams, important activities for Indigenous Peoples and local communities in the Pacific Northwest of the USA, due to a climate-driven algal bloom.
4. Declines in tourism and real estate values, associated with climate-driven harmful algal blooms, recorded in the USA, France, and England.

These disruptions have far-reaching impacts on the livelihoods and economies of various communities, particularly those heavily dependent on marine resources and coastal ecosystem services.

### Fine-Tuned `open-mistral-7b`

In [46]:
query_engine = index.as_query_engine(llm=ft_llm)

response = query_engine.query(questions[20])

display_response(response)

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


**`Final Response:`** Economic losses due to disruptions in ocean and coastal ecosystem services, partly attributable to climate change, have been recorded in various industries and regions. For instance, substantial economic losses for fisheries have been reported in Asia, North America, and South America due to climate-driven harmful algal blooms and marine pathogen outbreaks. A notable example is the 2016 event in Chile, which caused an estimated loss of 800 million USD in the farmed-salmon industry and led to regional government protests. Similarly, the recent closure of the Dungeness crab and razor clam fishery in the USA due to a climate-driven algal bloom affected 84% of surveyed residents from 16 California coastal communities.

In addition, changes in the ocean can impact tourism and real estate values. For example, declines in tourism and real estate values associated with climate-driven harmful algal blooms have been recorded in the USA, France, and England.

These examples illustrate the potential economic impacts on various industries and communities due to disruptions in ocean and coastal ecosystem services, which can be partly attributed to climate change.

As we can see, the fine-tuned model provides a more thorough response! This lines up with the increased faithfullness score from ragas, since the answer is more representative of the retrieved context.

## Conclusion

So, in conclusion, finetuning with only ~40 questions actually helped improve our eval scores!

**answer_relevancy: 0.0.8151 -> 0.8016**

The answer relevancy dips slightly but it's very small.

**faithfulness: 0.8360 -> 0.8924**

The faithfulness appears to have been improved! This mains the anwers given better fuffil the original question that was asked.